### OSM WD consistency och skapa karta

* Issue [#176](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/176) / [#180](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/180) / [#182](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/182)
* [this Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/182_WD_OSM.ipynb)

* [SAT Dashboard](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html)
   * [karta](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/182_WD_OSM_signs_latest.html) 


In [8]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-23 12:19


In [9]:
#!/usr/bin/env python3
"""
Generisk OSM↔Wikidata-koll via valfri SPARQL

Syfte
------
Skicka in *vilken SPARQL-fråga som helst* som returnerar ett ?item (Wikidata-objekt)
och **någon** form av OSM-id (node/way/relation) – t.ex. via standardegenskaperna
P10689 (node), P11693 (way), P402 (relation) eller som färdiga URL:er – så kontrollerar
skriptet att **motsvarande OSM-objekt pekar tillbaka** till Wikidata via `wikidata=Q…`.

Vad gör skriptet?
-----------------
1) Kör din SPARQL, tolkar resultatet och extraherar:
   - QID (från ?item)
   - OSM-ID:n (node/way/relation) från valfria fält/format.
2) Hämtar nämnda OSM-objekt direkt via Overpass (per id, ej bbox).
3) Validerar back-link:
   - ✅ **OK**: OSM har `wikidata`-tagg som *innehåller* samma QID (hanterar även semikolon-separerade listor).
   - ⚠️ **SAKNAS**: OSM saknar `wikidata`-tag helt.
   - ❌ **FEL**: OSM har `wikidata`, men den matchar **inte** QID.
4) Skriver ut **listor med klickbara länkar** för varje kategori och exporterar CSV.

Exempelanrop
------------
- `run_generic(SPARQL_QUERY)`

Notera
------
- Du kan fortfarande använda den tidigare `run()` för den hårdkodade SAT-frågan om du vill.
"""
from __future__ import annotations
import re
import math
import json
from typing import Dict, Iterable, List, Tuple

import pandas as pd
import requests

WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
OVERPASS_API = "https://overpass-api.de/api/interpreter"
USER_AGENT = "SAT-OSM-WD-Consistency/2.0 (contact: your-email@example.com)"

# -------------------------
# Hjälpfunktioner
# -------------------------

def make_headers() -> Dict[str, str]:
    return {"User-Agent": USER_AGENT}


def to_osm_prefixed_id(typ: str, osm_id: int) -> str:
    return {"node": "n", "way": "w", "relation": "r"}[typ] + str(int(osm_id))


def parse_osm_from_value(val: str) -> Tuple[str, int] | None:
    """Försök tolka ett OSM-id (typ, id) från godtycklig sträng/URL/tal.
    Stödjer t.ex. 12345, "12345", "https://www.openstreetmap.org/node/12345".
    Returnerar (typ, id) eller None.
    """
    if val is None:
        return None
    s = str(val).strip()

    # URL-format
    m = re.search(r"openstreetmap\.org\/(node|way|relation)\/(\d+)", s)
    if m:
        return m.group(1), int(m.group(2))

    # Rena tal: lämna typ okänd
    if s.isdigit():
        # Typ avgörs av fältnamnet – hanteras av anroparen
        return ("unknown", int(s))

    return None


# -------------------------
# Wikidata
# -------------------------

def fetch_wikidata_generic(sparql: str) -> pd.DataFrame:
    """Kör godtycklig SPARQL och extraherar QID och OSM-id:n ur valfria kolumner.
    Krav: ?item måste finnas och vara ett WD-objekt.
    Heuristik för OSM:
      - Leta kolumnnamn som innehåller något av: node|way|rel|osm|OSM
      - För varje sådan kolumn: tolka värdet som URL eller siffra.
      - Om typ är "unknown" – gissa typ utifrån kolumnnamn (node/way/relation).
    Returnerar DF med kolumner: qid, label (om finns), osm_type, osm_id
    (en rad per QID×OSM-referens). Om ingen OSM-referens hittas för en QID
    returneras ändå en rad utan OSM-typ/id för spårbarhet.
    """
    r = requests.get(WIKIDATA_SPARQL, params={"query": sparql, "format": "json"}, headers=make_headers())
    r.raise_for_status()
    rows = r.json()["results"]["bindings"]

    out_rows: List[Dict] = []
    for b in rows:
        item_uri = b.get("item", {}).get("value")
        if not item_uri:
            continue
        qid = item_uri.rsplit("/", 1)[-1]
        label = b.get("itemLabel", {}).get("value")

        # samla alla kandidater till OSM-kolumner
        osm_keys = [k for k in b.keys() if re.search(r"(node|way|rel|osm)", k, re.I) and k != "item"]

        found_any = False
        for k in osm_keys:
            raw = b[k].get("value")
            parsed = parse_osm_from_value(raw)
            if parsed:
                typ, oid = parsed
                if typ == "unknown":
                    # gissa från kolumnnamn
                    lk = k.lower()
                    if "node" in lk:
                        typ = "node"
                    elif "way" in lk:
                        typ = "way"
                    elif "rel" in lk:
                        typ = "relation"
                    else:
                        # kan ej avgöra – hoppa
                        continue
                out_rows.append({
                    "qid": qid, "label": label, "osm_type": typ, "osm_id": int(oid),
                    "osm_url": f"https://www.openstreetmap.org/{typ}/{int(oid)}",
                })
                found_any = True

        if not found_any:
            # lägg en rad utan OSM för spårbarhet
            out_rows.append({"qid": qid, "label": label, "osm_type": None, "osm_id": None, "osm_url": None})

    df = pd.DataFrame(out_rows).drop_duplicates()
    return df


# -------------------------
# Overpass
# -------------------------

def fetch_osm_by_ids(df_refs: pd.DataFrame) -> pd.DataFrame:
    """Hämta OSM-objekt per id för de rader som har osm_type/osm_id.
    Returnerar kolumner: osm_type, osm_id, url, wikidata_tag
    """
    subset = df_refs.dropna(subset=["osm_type", "osm_id"]).copy()
    if subset.empty:
        return pd.DataFrame(columns=["osm_type","osm_id","url","wikidata_tag"])   

    ids_by_type: Dict[str, List[int]] = {"node": [], "way": [], "relation": []}
    for _, r in subset.iterrows():
        t = r["osm_type"]
        if t in ids_by_type:
            ids_by_type[t].append(int(r["osm_id"]))

    def batched(lst: List[int], n: int = 200) -> Iterable[List[int]]:
        for i in range(0, len(lst), n):
            yield lst[i:i+n]

    rows: List[Dict] = []
    for t in ("node","way","relation"):
        if not ids_by_type[t]:
            continue
        for batch in batched(sorted(set(ids_by_type[t]))):
            ids_str = ",".join(str(i) for i in batch)
            q = f"""
            [out:json][timeout:50];
            {t}(id:{ids_str});
            out ids tags center;
            """.strip()
            rr = requests.post(OVERPASS_API, data={"data": q}, headers=make_headers())
            rr.raise_for_status()
            for el in rr.json().get("elements", []):
                tags = el.get("tags", {})
                rows.append({
                    "osm_type": t,
                    "osm_id": el.get("id"),
                    "url": f"https://www.openstreetmap.org/{t}/{el.get('id')}",
                    "wikidata_tag": tags.get("wikidata"),
                    "name": tags.get("name"),
                })

    return pd.DataFrame(rows)


# -------------------------
# Validering & utskrift
# -------------------------

def normalize_wikidata_values(val: str | None) -> List[str]:
    if not val:
        return []
    # Dela på semikolon/komma och trimma, ta bara sådant som ser ut som Q…
    parts = re.split(r"[;|,]", val)
    return [p.strip() for p in parts if re.match(r"^Q\d+$", p.strip(), re.I)]


def verify_backlinks(df_refs: pd.DataFrame, df_osm: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    # join på (osm_type, osm_id)
    key = ["osm_type","osm_id"]
    merged = pd.merge(df_refs.dropna(subset=key), df_osm, on=key, how="left", suffixes=("","_osm"))

    ok_rows = []
    missing_rows = []
    wrong_rows = []

    for _, r in merged.iterrows():
        qid = r["qid"]
        tag = r.get("wikidata_tag")
        if pd.isna(tag) or not tag:
            missing_rows.append(r)
        else:
            vals = normalize_wikidata_values(tag)
            if qid in vals:
                ok_rows.append(r)
            else:
                wrong_rows.append(r)

    return {
        "ok": pd.DataFrame(ok_rows),
        "missing": pd.DataFrame(missing_rows),
        "wrong": pd.DataFrame(wrong_rows),
        "all_refs": df_refs,
        "all_osm": df_osm,
    }


def print_link_lists(res: Dict[str, pd.DataFrame]):
    def print_group(title: str, df: pd.DataFrame):
        print(f"\n{title}")
        print("-" * len(title))
        if df.empty:
            print("(inget)")
            return
        # Grupp: per QID
        for qid, grp in df.groupby("qid"):
            print(f"{qid}:")
            for _, r in grp.iterrows():
                print(f"  {r['url']}")

    print_group("❌ OSM har wikidata men pekar fel", res["wrong"])        
    print_group("✅ OSM back-link OK (wikidata matchar QID)", res["ok"])    
    print_group("⚠️ OSM saknar wikidata-tag (lägg till QID)", res["missing"]) 


# -------------------------
# Publika körfunktioner
# -------------------------

def run_generic(sparql_query: str, export_prefix: str = "wd_osm_backlinks"):
    print("Kör SPARQL…")
    refs = fetch_wikidata_generic(sparql_query)
    print(f"Rader från SPARQL (QID×OSM-ref): {len(refs)}")

    print("Hämtar OSM per id…")
    osm = fetch_osm_by_ids(refs)
    print(f"OSM-objekt hämtade: {len(osm)}")

    res = verify_backlinks(refs, osm)

    # Exportera CSV
    for name, df in res.items():
        path = f"{export_prefix}_{name}.csv"
        df.to_csv(path, index=False)
        print(f"↳ sparade {path} ({len(df)} rader)")

    # Listrapport med klickbara länkar
    print_link_lists(res)



def run_sat_example():
    run_generic(SAT_SPARQL, export_prefix="sat_backlinks")


In [10]:
run_generic("""
SELECT ?item ?itemLabel ?OSMnode ?OSMway ?OSMrel WHERE {
  ?item wdt:P31 wd:Q1937027 .
  ?item wdt:P6104 wd:Q134294510 .
  OPTIONAL { ?item wdt:P10689 ?OSMway }
  OPTIONAL { ?item wdt:P11693 ?OSMnode }
  OPTIONAL { ?item wdt:P402   ?OSMrel }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,mul" }
}
""", export_prefix="sat_backlinks")


Kör SPARQL…
Rader från SPARQL (QID×OSM-ref): 66
Hämtar OSM per id…
OSM-objekt hämtade: 64
↳ sparade sat_backlinks_ok.csv (64 rader)
↳ sparade sat_backlinks_missing.csv (0 rader)
↳ sparade sat_backlinks_wrong.csv (2 rader)
↳ sparade sat_backlinks_all_refs.csv (66 rader)
↳ sparade sat_backlinks_all_osm.csv (64 rader)

❌ OSM har wikidata men pekar fel
--------------------------------
Q135916989:
  https://www.openstreetmap.org/node/12771834467
Q135973728:
  https://www.openstreetmap.org/node/12758924280

✅ OSM back-link OK (wikidata matchar QID)
-----------------------------------------
Q135010805:
  https://www.openstreetmap.org/node/12944560760
Q135045291:
  https://www.openstreetmap.org/node/12944833081
Q135045302:
  https://www.openstreetmap.org/node/12944798435
Q135045359:
  https://www.openstreetmap.org/node/12837567381
Q135045432:
  https://www.openstreetmap.org/node/12837571751
Q135045488:
  https://www.openstreetmap.org/node/12837532532
Q135045682:
  https://www.openstreetmap.org

In [11]:
#!/usr/bin/env python3
"""
Overpass→Wikidata konsistenskoll (med närhetssök-länkar & robust QID-validering)

Förbättringar jämfört med föregående version:
1) När OSM saknar `wikidata`-tag skapar skriptet en **Wikidata Query Service-länk**
   som gör en **närhetssökning (100 m)** kring OSM-objektets position.

2) QID-valideringen är nu robust:
   - Istället för att köra en ASK per QID (som kunde ge falskt fel p.g.a. throttling
     eller P31-kravet) gör vi **batchade SELECT**-frågor med `VALUES` och avgör
     existens utifrån vilka QID som faktiskt returneras av WDQS.
   - Hanterar semikolon-/komma-separerade `wikidata`-taggar.

Output:
- ✅ OSM med **minst en** giltig QID
- ⚠️ OSM **utan** `wikidata`-tag (med klickbar närhetssök-länk)
- ❌ OSM där **inga** QID i taggen finns i Wikidata

Dessutom exporteras CSV-filer med samma tre grupper + alla rådata.
"""
from __future__ import annotations
import re
import urllib.parse
from typing import Dict, Iterable, List, Set, Tuple

import pandas as pd
import requests

OVERPASS_API = "https://overpass-api.de/api/interpreter"
WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
USER_AGENT = "SAT-OSM-WD-Consistency/OverpassStart v2 (contact: your-email@example.com)"

DEFAULT_BBOX = (17.1, 58.8, 19.2, 60.6)
NEARBY_RADIUS_M = 100  # 100 meter


def make_headers() -> Dict[str, str]:
    return {"User-Agent": USER_AGENT}

# -------------------------
# Overpass
# -------------------------

def build_overpass_query(bbox: Tuple[float, float, float, float]) -> str:
    minlon, minlat, maxlon, maxlat = bbox
    return f"""
    [out:json][timeout:50];
    nwr["operator"="Stockholm Archipelago Trail"]["board_type"="hiking"]({minlat},{minlon},{maxlat},{maxlon});
    out ids tags center geom;
    """.strip()


def fetch_overpass(bbox=DEFAULT_BBOX) -> pd.DataFrame:
    q = build_overpass_query(bbox)
    r = requests.post(OVERPASS_API, data={"data": q}, headers=make_headers())
    r.raise_for_status()
    els = r.json().get("elements", [])
    rows = []
    for el in els:
        t = el.get("type")
        oid = el.get("id")
        tags = el.get("tags", {})
        # lat/lon – node har lat/lon direkt; way/relation har center
        lat = el.get("lat")
        lon = el.get("lon")
        if lat is None or lon is None:
            c = el.get("center") or {}
            lat = c.get("lat")
            lon = c.get("lon")
        rows.append({
            "osm_type": t,
            "osm_id": oid,
            "url": f"https://www.openstreetmap.org/{t}/{oid}",
            "wikidata_raw": tags.get("wikidata"),
            "name": tags.get("name"),
            "lat": lat,
            "lon": lon,
        })
    return pd.DataFrame(rows)

# -------------------------
# Wikidata-hjälp
# -------------------------

def normalize_qids(val: str | None) -> List[str]:
    if not val:
        return []
    parts = re.split(r"[;|,]", val)
    out = []
    for p in parts:
        s = p.strip()
        if re.match(r"^Q\d+$", s, re.I):
            out.append(s)
    return out


def wd_exist_bulk(qids: List[str], batch_size: int = 250) -> Set[str]:
    """Returnerar mängden QID som **existerar** i WDQS, batchad via VALUES.
    Enkla SELECT-frågor undviker problemen vi hade med ASK + P31.
    """
    existing: Set[str] = set()
    if not qids:
        return existing

    uniq = sorted(set(qids), key=lambda x: int(x[1:]))
    for i in range(0, len(uniq), batch_size):
        batch = uniq[i:i+batch_size]
        values = " ".join(f"wd:{q}" for q in batch)
        query = f"SELECT ?q WHERE {{ VALUES ?q {{ {values} }} }}"
        r = requests.get(WIKIDATA_SPARQL, params={"query": query, "format": "json"}, headers=make_headers())
        r.raise_for_status()
        for b in r.json()["results"]["bindings"]:
            uri = b["q"]["value"]  # https://www.wikidata.org/entity/Q123
            qid = uri.rsplit("/", 1)[-1]
            existing.add(qid)
    return existing


def build_wdqs_nearby_link(lat: float | None, lon: float | None, radius_m: int = NEARBY_RADIUS_M) -> str | None:
    if lat is None or lon is None:
        return None
    radius_km = radius_m / 1000.0
    sparql = f"""
#title: Nearby items (±{radius_m} m)
SELECT ?item ?itemLabel WHERE {{
  SERVICE wikibase:around {{
    ?item wdt:P625 ?location .
    bd:serviceParam wikibase:center "Point({lon} {lat})"^^geo:wktLiteral .
    bd:serviceParam wikibase:radius "{radius_km}" .
  }}
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en,mul" }}
}}
LIMIT 100
""".strip()
    return "https://query.wikidata.org/#" + urllib.parse.quote(sparql, safe="")

# -------------------------
# Klassificering
# -------------------------

def classify_osm(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df = df.copy()
    df["qids"] = df["wikidata_raw"].apply(normalize_qids)

    # Validera alla QIDs i en bulk mot WDQS
    all_qids = sorted({q for lst in df["qids"] for q in lst})
    existing = wd_exist_bulk(all_qids)

    # För varje rad: om qids och minst en i existing -> OK, annars WRONG; om inga qids -> MISSING
    ok_rows, missing_rows, wrong_rows = [], [], []
    for _, r in df.iterrows():
        qids = r["qids"]
        if not qids:
            r_dict = r.to_dict()
            r_dict["wdqs_nearby"] = build_wdqs_nearby_link(r.get("lat"), r.get("lon"))
            missing_rows.append(r_dict)
        else:
            if any(q in existing for q in qids):
                ok_rows.append(r.to_dict())
            else:
                wrong_rows.append(r.to_dict())

    ok = pd.DataFrame(ok_rows)
    missing = pd.DataFrame(missing_rows)
    wrong = pd.DataFrame(wrong_rows)

    return df, ok, missing, wrong

# -------------------------
# Utskrift
# -------------------------

def print_links(ok: pd.DataFrame, missing: pd.DataFrame, wrong: pd.DataFrame):
    print("\n✅ OSM med minst en giltig QID:")
    if ok.empty:
        print("  (inget)")
    else:
        for _, r in ok.iterrows():
            print(f"  {r['url']} → {r['wikidata_raw']}")

    print("\n⚠️ OSM utan wikidata-tag (närhetssök 100 m):")
    if missing.empty:
        print("  (inget)")
    else:
        for _, r in missing.iterrows():
            link = r.get("wdqs_nearby") or "(saknar koordinat – ingen närhetssökning)"
            print(f"  {r['url']}  →  {link}")

    print("\n❌ OSM med ogiltig/felaktig wikidata-tag (inga QID existerar i WD):")
    if wrong.empty:
        print("  (inget)")
    else:
        for _, r in wrong.iterrows():
            print(f"  {r['url']} → {r['wikidata_raw']}")

# -------------------------
# Körning
# -------------------------

def run_from_overpass(bbox=DEFAULT_BBOX, export_prefix="osm_overpass_backlinks_v2"):
    print("Hämtar OSM via Overpass…")
    osm = fetch_overpass(bbox)
    print(f"OSM-element: {len(osm)}")

    df_all, ok, missing, wrong = classify_osm(osm)

    # Export
    df_all.to_csv(f"{export_prefix}_all.csv", index=False)
    ok.to_csv(f"{export_prefix}_ok.csv", index=False)
    missing.to_csv(f"{export_prefix}_missing.csv", index=False)
    wrong.to_csv(f"{export_prefix}_wrong.csv", index=False)

    print_links(ok, missing, wrong)


if __name__ == "__main__":
    run_from_overpass()


Hämtar OSM via Overpass…
OSM-element: 60

✅ OSM med minst en giltig QID:
  https://www.openstreetmap.org/node/863394962 → Q135057972
  https://www.openstreetmap.org/node/6730659174 → Q135062583
  https://www.openstreetmap.org/node/12758854059 → Q135063085
  https://www.openstreetmap.org/node/12758872849 → Q135062808
  https://www.openstreetmap.org/node/12758913300 → Q135063320
  https://www.openstreetmap.org/node/12758914941 → Q135065340
  https://www.openstreetmap.org/node/12758929382 → Q135063491
  https://www.openstreetmap.org/node/12759045298 → Q135056610
  https://www.openstreetmap.org/node/12759055196 → Q135065022
  https://www.openstreetmap.org/node/12759062051 → Q135055618
  https://www.openstreetmap.org/node/12759068092 → Q135054293
  https://www.openstreetmap.org/node/12759068968 → Q135054141
  https://www.openstreetmap.org/node/12759068970 → Q135054522
  https://www.openstreetmap.org/node/12759074801 → Q135053596
  https://www.openstreetmap.org/node/12759101764 → Q135046665


### Skapa karta 

In [20]:
from string import Template
from datetime import datetime as dt
import html  # vi använder html.escape här

def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
    offset_px=(10, 54),  # (top, left) – 54px för att hamna snällt bredvid zoom
):
    """Superenkel, robust About-box som alltid syns."""
    if created_date is None:
        created_date = dt.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"
    top, left  = offset_px
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    links = [
        ("SAT Dashboard", "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),
    ]
    links_html = "".join(
        f'<div><a href="{html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {html.escape(t)}</a></div>'
        for t, u in links
    )

    tpl = Template(r"""
<style>
  .sat-about {
    position: fixed; z-index: 10000;
    top: ${top}px; left: ${left}px;
    background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    min-width: 240px; max-width: 320px; pointer-events: auto;
  }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
</style>

<div id="${box_id}" class="sat-about ${collapsed_class}">
  <div id="${header_id}" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span><span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="${issue_url}" target="_blank">#${issue_number}</a>&nbsp;&nbsp; Map: ${map_name}</div>
    <div>Created: ${created_date}</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">${links_html}</div>
  </div>
</div>

<script>
(function(){
  var boxId = "${box_id}";
  var hdrId = "${header_id}";
  var storageKey = "satAboutCollapsed_${map_dom_id}_#${issue_number}";

  function setCollapsed(box, collapsed) {
    if (!box) return;
    if (collapsed) box.classList.add("sat-about-collapsed");
    else box.classList.remove("sat-about-collapsed");
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function init(){
    var box = document.getElementById(boxId);
    var hdr = document.getElementById(hdrId);
    if (!box || !hdr) return;

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      setCollapsed(box, !box.classList.contains("sat-about-collapsed"));
    });
  }

  if (document.readyState === "loading") {
    document.addEventListener("DOMContentLoaded", init);
  } else {
    init();
  }
})();
</script>
""")

    html_code = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        issue_number=issue_number,
        issue_url=issue_url,
        map_name=html.escape(map_name),
        created_date=created_date,
        links_html=links_html,
        collapsed_class=collapsed_class,
        map_dom_id=map_dom_id,
        top=top, left=left,
    )
    m.get_root().html.add_child(folium.Element(html_code))


In [23]:
import os, json, datetime, html
import requests
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap, MeasureControl, MousePosition

# =========================
# KONFIG
# =========================
SAT_GEOJSON_PATH   = "SAT_full.geojson"   # din fil
OUTPUT_PREFIX      = "182_WD_OSM_signs"
DASHBOARD_URL      = "https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/SAT_ALL_IN_ONE_142_3_dashboard_latest.html"
GITHUB_LABEL_URL   = "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue%20label%3A%22SAT%20Informationsskyltar%22"
WANDER_TIME_URL    = "https://www.wanderndeluxe.de/en/calculate-hiking-time-distance-altitude/"
FETCH_FROM_WD      = True                 # sätt False om du redan har signs_wd.csv lokalt

# SPARQL: skyltar med koordinat, bild, officiell web (sv/en), delsträcka + längd + OSM node
SPARQL = r"""
SELECT ?item ?itemLabel ?OSMnode ?coord ?img ?trail ?trailLabel ?lengthkm ?officiellWebswe ?officiellWeben WHERE {
  ?item wdt:P31 wd:Q1937027 .
  ?item wdt:P6104 wd:Q134294510 .
  ?item wdt:P625 ?coord .
  ?item wdt:P2789 ?trail .
  ?item wdt:P18 ?img.
  ?trail wdt:P31 wd:Q2143825.
  ?trail wdt:P2043 ?lengthkm.
  ?item wdt:P11693 ?OSMnode.

  OPTIONAL {
    ?trail p:P856 ?sweStatement .
    ?sweStatement ps:P856 ?officiellWebswe .
    ?sweStatement pq:P407 wd:Q9027 .
  }
  OPTIONAL {
    ?trail p:P856 ?enStatement .
    ?enStatement ps:P856 ?officiellWeben .
    ?enStatement pq:P407 wd:Q1860 .
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en,mul" }
}
"""

# =========================
# HJÄLPARE
# =========================
def fetch_wd_signs_to_csv(out_csv="signs_wd.csv"):
    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "SAT-map-builder/1.1 (contact: you@example.com)"
    }
    r = requests.get(url, params={"query": SPARQL}, headers=headers, timeout=80)
    r.raise_for_status()
    data = r.json()
    rows = []
    for b in data["results"]["bindings"]:
        gv = lambda k: b[k]["value"] if k in b else None
        rows.append({
            "item": gv("item"),
            "itemLabel": gv("itemLabel"),
            "OSMnode": gv("OSMnode"),
            "coord": gv("coord"),
            "img": gv("img"),
            "trail": gv("trail"),
            "trailLabel": gv("trailLabel"),
            "lengthkm": gv("lengthkm"),
            "officiellWebswe": gv("officiellWebswe"),
            "officiellWeben": gv("officiellWeben"),
        })
    df = pd.DataFrame(rows)
    # parse coord "Point(lon lat)"
    lats, lons = [], []
    for v in df["coord"].astype(str):
        lat = lon = None
        if v.startswith("Point(") and v.endswith(")"):
            parts = v[6:-1].split()
            if len(parts) == 2:
                try:
                    lon = float(parts[0]); lat = float(parts[1])
                except:
                    pass
        lats.append(lat); lons.append(lon)
    df["lat"] = lats; df["lon"] = lons
    df.to_csv(out_csv, index=False)
    return df

def bounds_from_geojson(gj):
    minlat=minlon= 1e9
    maxlat=maxlon=-1e9
    def add(coords):
        nonlocal minlat, minlon, maxlat, maxlon
        if isinstance(coords[0], (float,int)):
            lon, lat = coords
            minlat = min(minlat, lat); maxlat = max(maxlat, lat)
            minlon = min(minlon, lon); maxlon = max(maxlon, lon)
        else:
            for c in coords:
                add(c)
    for feat in gj.get("features", []):
        geom = feat.get("geometry", {})
        add(geom.get("coordinates", []))
    if minlat == 1e9:
        return (59.3, 18.0, 59.6, 19.2)
    return (minlat, minlon, maxlat, maxlon)

def h2hmm(hours_float: float) -> str:
    m = int(round(hours_float * 60))
    h, mm = divmod(m, 60)
    return f"{h}:{mm:02d}"

def hiking_time(distance_km: float, ascent_m: float=None, descent_m: float=None) -> (str, str):
    """
    Om vi har höjddata → Schweiziska formeln:
      T = max(Th, Tv) + 0.5 * min(Th, Tv)
      Th = distance / 4 km/h
      Tv = ascent/300 + descent/500 (h)
    Annars fallback: Th = distance / 4 km/h
    Returnerar (tid_hh:mm, "förklarings-text")
    """
    if distance_km is None:
        return ("—", "Saknar sträcka; ingen beräkning.")
    Th = float(distance_km) / 4.0
    if ascent_m is not None and descent_m is not None:
        Tv = float(ascent_m)/300.0 + float(descent_m)/500.0
        T = max(Th, Tv) + 0.5 * min(Th, Tv)
        explain = (f"⏱️ Tid enligt CH-formel: max(Th, Tv) + 0.5·min(Th, Tv). "
                   f"Th= sträcka/4 km/h = {h2hmm(Th)}, "
                   f"Tv= +{ascent_m}/300 + -{descent_m}/500 h = {h2hmm(Tv)}. "
                   f"Källa: <a href='{WANDER_TIME_URL}' target='_blank'>wanderndeluxe</a>.")
        return (h2hmm(T), explain)
    else:
        explain = (f"⏱️ Förenklad tid utan höjddata: Th = sträcka/4 km/h = {h2hmm(Th)}. "
                   f"Se CH-formeln här: <a href='{WANDER_TIME_URL}' target='_blank'>wanderndeluxe</a>.")
        return (h2hmm(Th), explain)

def safe(s):
    return None if (s is None or (isinstance(s, float) and pd.isna(s))) else str(s)

# =========================
# 1) Hämta/läs skyltar
# =========================
if FETCH_FROM_WD:
    print("Hämtar skyltar från Wikidata…")
    df_signs = fetch_wd_signs_to_csv("signs_wd.csv")
else:
    if not os.path.exists("signs_wd.csv"):
        raise FileNotFoundError("Hittar inte signs_wd.csv – kör FETCH_FROM_WD=True eller lägg filen bredvid scriptet.")
    df_signs = pd.read_csv("signs_wd.csv")

print(f"Skyltar: {len(df_signs)}")

# =========================
# 2) Läs SAT_full.geojson
# =========================
if not os.path.exists(SAT_GEOJSON_PATH):
    raise FileNotFoundError(f"Saknar {SAT_GEOJSON_PATH}.")
with open(SAT_GEOJSON_PATH, "r", encoding="utf-8") as f:
    sat_geo = json.load(f)

# =========================
# 3) Initiera karta
# =========================
minlat, minlon, maxlat, maxlon = bounds_from_geojson(sat_geo)
center = [(minlat+maxlat)/2, (minlon+maxlon)/2]

m = folium.Map(location=center, zoom_start=10, tiles="CartoDB positron", control_scale=True)
Fullscreen().add_to(m)
MiniMap(toggle_display=True, position="bottomleft").add_to(m)
MeasureControl(position='topleft', primary_length_unit='kilometers', secondary_length_unit='meters').add_to(m)
MousePosition(position='bottomright', separator=' | ', num_digits=5, prefix='Lat/Lon:').add_to(m)

# 4) SAT-spår
sat_fg = folium.FeatureGroup(name="SAT spår (SAT_full.geojson)", show=True)
folium.GeoJson(
    sat_geo,
    name="SAT_full",
    style_function=lambda feat: {"color":"#e4572e","weight":5,"opacity":0.9},
).add_to(sat_fg)
sat_fg.add_to(m)

# 5) Skylter (ingen clustering → alla ikoner visas direkt)
signs_fg = folium.FeatureGroup(name="Informationsskyltar", show=True)
for _, row in df_signs.iterrows():
    lat, lon = row.get("lat"), row.get("lon")
    if pd.isna(lat) or pd.isna(lon): 
        continue

    item        = safe(row.get("item"))
    itemLabel   = safe(row.get("itemLabel")) or "Informationsskylt"
    img         = safe(row.get("img"))
    trail       = safe(row.get("trail"))
    trailLabel  = safe(row.get("trailLabel"))
    lengthkm    = safe(row.get("lengthkm"))
    lengthkm_v  = float(lengthkm) if lengthkm not in (None, "None", "") else None

    # har du ascent_m/descent_m i CSV? plocka dem; annars blir None
    ascent_m    = None
    descent_m   = None

    time_hhmm, explain = hiking_time(lengthkm_v, ascent_m, descent_m)

    # Länkar
    wd_url      = item if (item and item.startswith("http")) else (f"https://www.wikidata.org/wiki/{item.split('/')[-1]}" if item else None)
    trail_url   = trail if (trail and trail.startswith("http")) else (f"https://www.wikidata.org/wiki/{trail.split('/')[-1]}" if trail else None)
    osmnode     = safe(row.get("OSMnode"))
    osm_url     = f"https://www.openstreetmap.org/node/{osmnode}" if osmnode else None
    swe         = safe(row.get("officiellWebswe"))
    en          = safe(row.get("officiellWeben"))

    # Snyggare bildskalning (responsiv)
    img_html = ""
    if img:
        img_html = f"""
        <div style="margin:8px 0;">
          <div style="
            width:100%;
            max-width:320px;
            aspect-ratio: 4 / 3;
            overflow:hidden;
            border-radius:10px;
            box-shadow:0 6px 18px rgba(0,0,0,.18);
          ">
            <img src="{html.escape(img)}" style="width:100%; height:100%; object-fit:cover;" referrerpolicy="no-referrer">
          </div>
        </div>
        """
    # Länkar i tight lista
    links = []
    if swe:      links.append(f"🌐 <a href='{html.escape(swe)}' target='_blank'>Officiell webb (sv)</a>")
    if en:       links.append(f"🌐 <a href='{html.escape(en)}' target='_blank'>Official website (en)</a>")
    if wd_url:   links.append(f"🧠 <a href='{html.escape(wd_url)}' target='_blank'>Wikidata</a>")
    if osm_url:  links.append(f"🗺️ <a href='{html.escape(osm_url)}' target='_blank'>OSM node</a>")
    if trail_url:links.append(f"🧭 <a href='{html.escape(trail_url)}' target='_blank'>Delsträcka (WD)</a>")
    links.append(f"🔗 <a href='{html.escape(DASHBOARD_URL)}' target='_blank'>SAT Dashboard</a>")
    links.append(f"🔗 <a href='{html.escape(GITHUB_LABEL_URL)}' target='_blank'>GITHUB – SAT Informationsskyltar</a>")

    links_html = "<ul style='list-style:none; padding:0; margin:6px 0 0 0; font-size:13px; line-height:1.4;'>"
    for l in links:
        links_html += f"<li style='margin:2px 0;'>{l}</li>"
    links_html += "</ul>"

    html_popup = f"""
    <div style="font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; max-width: 330px; line-height:1.35;">
      <div style="font-weight:600; font-size:15px; margin-bottom:4px;">{html.escape(itemLabel)}</div>
      <div style="font-size:11px; opacity:0.7; margin-bottom:6px;">tourism=information • information=guidepost</div>
      {img_html}
      <div style="font-size:13px; margin:2px 0;">⏱️ <b>Vandringstid:</b> {time_hhmm}</div>
      <div style="font-size:13px; margin:2px 0;">📏 <b>Sträcka:</b> {f"{lengthkm_v:.1f} km" if lengthkm_v is not None else "—"}</div>
      <div style="font-size:13px; margin:2px 0;">🧭 <b>Delsträcka:</b> {html.escape(trailLabel) if trailLabel else ""}</div>
      <div style="margin-top:4px; font-size:11px; opacity:0.85;">
        {explain}
      </div>
      {links_html}
    </div>
    """

    folium.Marker(
        location=[float(lat), float(lon)],
        icon=folium.Icon(icon="info-sign", color="blue"),
        popup=folium.Popup(html_popup, max_width=380)
    ).add_to(signs_fg)

signs_fg.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

add_about_box(m, issue_number=182, map_name="SAT skyltar")

# 6) spara
os.makedirs("output", exist_ok=True)  # se till att katalogen finns

ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")

m.save(out_ts)
m.save(out_latest)

print("Klar:")
print("  ", out_ts)
print("  ", out_latest)


Hämtar skyltar från Wikidata…
Skyltar: 66
Klar:
   output/182_WD_OSM_signs_20250923_124110.html
   output/182_WD_OSM_signs_latest.html


In [22]:
end_time = time.time()
duration = end_time - start_time
print(f"Finished in {duration:.2f} seconds.")


Finished in 712.11 seconds.
